#Monthly milk production: pounds per cow. Jan 1962 - Dec 1975

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Activation,Dense,Dropout
%matplotlib inline

In [ ]:
df = pd.read_csv('/content/monthly-milk-production.csv',index_col='Month')
df.index = pd.to_datetime(df.index)
df

In [ ]:
df.plot()

In [ ]:
scaler = MinMaxScaler()

array = []
train_data = []
train_labels = []

for i in range(len(df)):
    array.append(df.iloc[i]['Milk Production'])

array = np.array(array).reshape(-1,1)
array = scaler.fit_transform(array)
array

In [ ]:
k = 0
for i in range(len(array)):
    try:
        train_data.append(array[12*k:12*(k+1)])
        train_labels.append(array[12*(k+1)])
        k+=1
    except:
        break

In [ ]:
train_data = np.squeeze(train_data)
train_data.shape

In [ ]:

train_labels = np.array(train_labels)
train_labels.shape

In [ ]:
train_data = train_data[:len(train_labels)]
train_data = np.expand_dims(train_data,1)
train_data.shape

In [ ]:
train_data[1]

In [ ]:
train_labels[1]

In [ ]:
model = Sequential()

model.add(LSTM(250,input_shape=(1,12)))
model.add(Dropout(0.5))


model.add(Dense(250,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(12,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='relu'))

model.compile(loss='mean_squared_error',optimizer='adam')

model.summary()

In [ ]:
E = 1000

H = model.fit(train_data,train_labels,epochs=E)

In [ ]:
epochs = range(0,E)
loss = H.history['loss']
plt.plot(epochs,loss)

In [ ]:
preds = scaler.inverse_transform(model.predict(train_data))
plt.plot(range(0,13),preds,label='our predictions')
plt.plot(range(0,13),scaler.inverse_transform(train_labels),label='real values')
plt.legend()

#Now we have trained our model which takes previous 12 month data as input and predicts next month milk production. We will carry forward this method and predict production for next whole year.

In [ ]:
seed = array[-12:]
seed

In [ ]:
seed.shape

In [ ]:
for _ in range(12):
    curr_12_months = seed[-12:]
    curr_12_months = np.squeeze(curr_12_months)
    curr_12_months = np.expand_dims(curr_12_months,0)
    curr_12_months = np.expand_dims(curr_12_months,0)
    pred = model.predict(curr_12_months)
    seed = np.append(seed,pred)

In [ ]:
seed

In [ ]:
next_year_prediction = scaler.inverse_transform(seed[-12:].reshape(-1,1))
plt.plot(range(0,12),next_year_prediction,marker='*',markerfacecolor='red',markersize=10)